# Revista de Administração, Contabilidade e Sustentabilidade – REUNIR

## Bibliotecas importadas

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

## Funções básicas

In [2]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [3]:
def listar_links_pagina(url_search):
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find_all('h2', {'class': 'title'})
    a_links = []
    for item in results:
        try:
            remover_links = 'editorial' in item.find('a').text.lower()
            remover_links = remover_links or 'folha de rosto' in item.find('a').text.lower()
            remover_links = remover_links or 'dados da edição ' in item.find('a').text.lower()
            if not remover_links:
                a_links.append(item.find('a')['href'])
        except:
            pass
    return a_links

In [4]:
def listar_links(url_search_text, query, pg_num=1):
    a_links = [] 
    while True:
        url_search = url_search_text.replace('__query__', query).replace('__pg_num__', str(pg_num))
        tmp = listar_links_pagina(url_search)
        if len(tmp) != 0:
            pg_num += 1
            a_links += tmp
        else:
            break
    return a_links

In [5]:
url_search_text = 'https://reunir.revistas.ufcg.edu.br/index.php/uacc/search/search?query=__query__&searchJournal=1&authors=&title=&abstract=&galleyFullText=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&orderBy=score&orderDir=desc&searchPage=__pg_num__#results'

links = listar_links(url_search_text, 'contabilidade+ambiental', pg_num=1)

link = links[0]

link


'https://reunir.revistas.ufcg.edu.br/index.php/uacc/article/view/627'

In [6]:
links

['https://reunir.revistas.ufcg.edu.br/index.php/uacc/article/view/627',
 'https://reunir.revistas.ufcg.edu.br/index.php/uacc/article/view/153',
 'https://reunir.revistas.ufcg.edu.br/index.php/uacc/article/view/65',
 'https://reunir.revistas.ufcg.edu.br/index.php/uacc/article/view/29',
 'https://reunir.revistas.ufcg.edu.br/index.php/uacc/article/view/108',
 'https://reunir.revistas.ufcg.edu.br/index.php/uacc/article/view/102',
 'https://reunir.revistas.ufcg.edu.br/index.php/uacc/article/view/394',
 'https://reunir.revistas.ufcg.edu.br/index.php/uacc/article/view/386',
 'https://reunir.revistas.ufcg.edu.br/index.php/uacc/article/view/281',
 'https://reunir.revistas.ufcg.edu.br/index.php/uacc/article/view/63',
 'https://reunir.revistas.ufcg.edu.br/index.php/uacc/article/view/740',
 'https://reunir.revistas.ufcg.edu.br/index.php/uacc/article/view/1031',
 'https://reunir.revistas.ufcg.edu.br/index.php/uacc/article/view/72',
 'https://reunir.revistas.ufcg.edu.br/index.php/uacc/article/view/1

In [7]:
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

In [8]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [9]:
def limpar_afiliacao(afiliacao):
    afiliacao = ' '.join(afiliacao.split())
    afiliacao = afiliacao.split(' - ')[0]
    afiliacao = afiliacao.split('(')[0]
    afiliacao = afiliacao.split('/')[0]
    return afiliacao.strip()

In [10]:
def captura_titulo(soup):
    try:
        title = soup.find('h1', {'class': 'page_title'}).text
        title = ' '.join(title.split())
    except:
        title = ''
    return {'Título': title}

In [11]:
captura_titulo(soup)

{'Título': 'ALICERCE À MENSURAÇÃO DAS AUTUAÇÕES AMBIENTAIS NO ESTADO DO CEARÁ: UM ESTUDO APLICADO NAS INDÚSTRIAS DE MINERAIS NÃO METÁLICOS'}

In [12]:
def captura_data(soup):
    try:
        date_tag = soup.find('div', {'class': 'item published'})
        date = date_tag.find('div', {'class': 'value'}).text
        date = ' '.join(date.split())
    except:
        date = ''
    return {'Data de Publicação': date[:10]}

In [13]:
captura_data(soup)

{'Data de Publicação': '2019-01-01'}

In [14]:
def captura_resumo(soup):
    try:
        abstract_section = soup.find('section', {'class': 'item abstract'})
        abstract = abstract_section.text
        abstract = abstract.replace('Resumo:', '')
        abstract = abstract.replace('Resumo', '').strip()
        #abstract = ' '.join(abstract.split())
    except:
        abstract = ''
    return {'Resumo': abstract}

In [15]:
captura_resumo(soup)

{'Resumo': 'Esta pesquisa tem como objetivo geral analisar os fundamentos utilizados na mensuração dos autos de infração ambiental aplicados nas indústrias de minerais não metálicos do Estado do Ceará, no período de janeiro de 2011 a junho de 2012. Os dados foram coletados em de dezembro de 2014, no acervo do CAOMACE. Em posse dos autos, recorreu-se à análise de conteúdo como técnica qualitativa de coleta dos dados de 58 autos de infração. Os resultados mostraram que a utilização de produtos florestais sem licença e o funcionamento de fábricas poluidoras sem licença ambiental são tipos de infração de maior ocorrência; que há casos em que a aplicação de multas fixas não corresponde ao estabelecido legalmente; que a maioria das infrações abertas foi valorada dentro dos limites legais, sem detalhamento técnico da metodologia utilizada e, ainda, que os critérios utilizados como parâmetros para o cálculo das multas abertas foram cinco (o porte do empreendimento, o potencial poluidor degrada

In [16]:
def captura_palavras_chave(soup):
    try:
        kw_section = soup.find('section', {'class': 'item keywords'})
        kw = kw_section.text.replace('Palavras-chave:', '')
        kw = ' '.join(kw.split())
        kw = kw.replace(',', '.').replace(';', '.')
    except:
        kw = ''
    if kw == '':
        try:
            kw_section = soup.find('section', {'class': 'item abstract'})
            p_list = kw_section.find_all('p')
            for p in p_list:
                if 'Palavras-chave' in p.text:
                    kw_full = p.text
                    kw = kw_full.replace('Palavras-chave', '').replace(':', '')
                    kw = ' '.join(kw.split())
                    kw = kw.replace(',', '.').replace(';', '.')
        except:
            pass
    return {'Palavras-chave': kw}

In [17]:
captura_palavras_chave(soup)

{'Palavras-chave': 'Indústrias de minerais não metálicos. Autos de infração. Multas ambientais'}

In [18]:
def captura_autores(soup):
    autores = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        autores = authors_ul.find_all('span', {'class': 'name'})
        autores = [' '.join(a.text.split()) for a in autores]
    except:
        pass
    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [19]:
captura_autores(soup)

{'Qtd. de Autores': 4,
 'Autor 1': 'Ivaneide Ferreira Farias',
 'Autor 2': 'Jackeline Lucas Souza',
 'Autor 3': 'Miguel Carioca Neto',
 'Autor 4': 'Liliane Maria Ramalho de Castro Siqueira',
 'Autor 5': '',
 'Autor 6': ''}

In [20]:
def captura_afiliacao(soup):
    afiliacao = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        afiliacao = authors_ul.find_all('span', {'class': 'affiliation'})
        afiliacao = [' '.join(a.text.split()) for a in afiliacao]
        afiliacao = [limpar_afiliacao(a) for a in afiliacao]
    except:
        pass
    afiliacao = completar_lista(afiliacao)
    
    key_affiliation = [f"Afiliação {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, afiliacao))

In [21]:
captura_afiliacao(soup)

{'Afiliação 1': 'Universidade Federal do Ceará',
 'Afiliação 2': 'Universidade Federal do Ceará',
 'Afiliação 3': 'Universidade Federal Rural do Semi-Árido',
 'Afiliação 4': 'Universidade Federal do Ceará',
 'Afiliação 5': '',
 'Afiliação 6': ''}

In [22]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'Revista': 'Revista de Administração, Contabilidade e Sustentabilidade – REUNIR'}
    url_search_text = 'https://reunir.revistas.ufcg.edu.br/index.php/uacc/search/search?query=__query__&searchJournal=1&authors=&title=&abstract=&galleyFullText=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&orderBy=score&orderDir=desc&searchPage=__pg_num__#results'
    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links(url_search_text, query)
        
    resultados = []
    for link in a_links:
        link_dict = {'Url Artigo': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        dict_artigo = {
            **captura_data(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%Y-%m-%d')
    return df

## Pesquisa por artigos

In [23]:
key_words_list = ['contabilidade ambiental', 'balanço social', 'relato integrado', 'nbct 15']
df = realizar_consulta(key_words_list)
df.head(3)

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2019-01-01,ALICERCE À MENSURAÇÃO DAS AUTUAÇÕES AMBIENTAIS...,Esta pesquisa tem como objetivo geral analisar...,Indústrias de minerais não metálicos. Autos de...,4,Ivaneide Ferreira Farias,Jackeline Lucas Souza,Miguel Carioca Neto,Liliane Maria Ramalho de Castro Siqueira,,,Universidade Federal do Ceará,Universidade Federal do Ceará,Universidade Federal Rural do Semi-Árido,Universidade Federal do Ceará,,,https://reunir.revistas.ufcg.edu.br/index.php/...,"Revista de Administração, Contabilidade e Sust..."
1,2014-08-07,AVALIAÇÃO DE FUNÇÕES DE GOVERNOS ESTADUAIS E D...,Esta pesquisa tem o objetivo de levantar opini...,,4,Maurício Corrêa da Silva,Fábia Jaiany Viana de Souza,Aneide Oliveira Araujo,José Dionísio Gomes da Silva,,,Universidade Federal do Rio Grande do Norte,UFRN,UFRN,,,,https://reunir.revistas.ufcg.edu.br/index.php/...,"Revista de Administração, Contabilidade e Sust..."
2,2012-08-07,ANÁLISE DA INTERAÇÃO DA CONTABILIDADE AMBIENTA...,Hoje a contabilidade assume um papel intermedi...,Contabilidade Ambiental. Desenvolvimento Suste...,2,Fabiane Tubino Garcia,Ariel Behr,,,,,Universidade Federal do Pampa,Universidade Federal do Pampa,,,,,https://reunir.revistas.ufcg.edu.br/index.php/...,"Revista de Administração, Contabilidade e Sust..."


In [24]:
df.to_csv('../data/REUNIR.csv', index=False)

In [25]:
df

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2019-01-01,ALICERCE À MENSURAÇÃO DAS AUTUAÇÕES AMBIENTAIS...,Esta pesquisa tem como objetivo geral analisar...,Indústrias de minerais não metálicos. Autos de...,4,Ivaneide Ferreira Farias,Jackeline Lucas Souza,Miguel Carioca Neto,Liliane Maria Ramalho de Castro Siqueira,,,Universidade Federal do Ceará,Universidade Federal do Ceará,Universidade Federal Rural do Semi-Árido,Universidade Federal do Ceará,,,https://reunir.revistas.ufcg.edu.br/index.php/...,"Revista de Administração, Contabilidade e Sust..."
1,2014-08-07,AVALIAÇÃO DE FUNÇÕES DE GOVERNOS ESTADUAIS E D...,Esta pesquisa tem o objetivo de levantar opini...,,4,Maurício Corrêa da Silva,Fábia Jaiany Viana de Souza,Aneide Oliveira Araujo,José Dionísio Gomes da Silva,,,Universidade Federal do Rio Grande do Norte,UFRN,UFRN,,,,https://reunir.revistas.ufcg.edu.br/index.php/...,"Revista de Administração, Contabilidade e Sust..."
2,2012-08-07,ANÁLISE DA INTERAÇÃO DA CONTABILIDADE AMBIENTA...,Hoje a contabilidade assume um papel intermedi...,Contabilidade Ambiental. Desenvolvimento Suste...,2,Fabiane Tubino Garcia,Ariel Behr,,,,,Universidade Federal do Pampa,Universidade Federal do Pampa,,,,,https://reunir.revistas.ufcg.edu.br/index.php/...,"Revista de Administração, Contabilidade e Sust..."
3,2015-12-14,CONTABILIDADE AMBIENTAL: A EVIDENCIAÇÃO NAS DE...,Para garantir longevidade e sucesso das instit...,Contabilidade Ambiental. Gestão Ambiental. Evi...,2,Débora Pool da Silva Freitas,Walter Nunes Oleiro,,,,,FURG- UNIVERSIDADE FEDERAL DO RIO GRANDE,FURG,,,,,https://reunir.revistas.ufcg.edu.br/index.php/...,"Revista de Administração, Contabilidade e Sust..."
4,2014-02-26,DISCLOSURE AMBIENTAL: UM ESTUDO NAS EMPRESAS D...,A necessidade de demonstrar as informações ger...,Contabilidade Ambiental. Norma Brasileira de C...,4,Eliedna de Sousa Barbosa,Ádria Tayllo Alves Oliveira,Roseane Patrícia de Araújo Silva,Isabelle Carlos Campos Rezende,,,Universidade Estadual da Paraíba,Universidade Estadual da Paraíba,Universidade Estadual da Paraíba,Universidade Federal da Paraíba,,,https://reunir.revistas.ufcg.edu.br/index.php/...,"Revista de Administração, Contabilidade e Sust..."
5,2013-02-25,O PRODUTO VERDE É SEMPRE “VERDE”? UMA ANÁLISE ...,"Atualmente, face aos problemas ambientais caus...",Ecodesign. Produto verde. Ecodesign PILOT. Tei...,2,Isabel Joselita Barbosa da Rocha Alves,Lúcia Santana de Freitas,,,,,Universidade Estadual da Paraíba,Universidade Federal de Campina Grande. Unidad...,,,,,https://reunir.revistas.ufcg.edu.br/index.php/...,"Revista de Administração, Contabilidade e Sust..."
6,2016-09-01,O processo de desertificação no Município de A...,O presente trabalho analisa o quadro ambiental...,Região do Seridó. Acari/RN. desertificação. ca...,3,Girlene Edson de Oliveira Amaro,Maria Luiza de Medeiros Galvão,Kettrin Farias Bem Maracajá,,,,Especialista em Educação Ambiental e Geografia...,Doutoranda no Programa de Planejamento Energét...,"Universidade Federal de Campina Grande, Unidad...",,,,https://reunir.revistas.ufcg.edu.br/index.php/...,"Revista de Administração, Contabilidade e Sust..."
7,2016-04-29,MÉTRICA DE VALORAÇÃO AMBIENTAL: UMA PERCEPÇÃO ...,A pesquisa objetivou analisar a percepção dos ...,Ativos Ambientais. Heritages Assets. Método do...,2,Fernanda Jaqueline Lopes,Fátima de Souza Freire,,,,,Universidade de Brasília,Universidade de Brasília,,,,,https://reunir.revistas.ufcg.edu.br/index.php/...,"Revista de Administração, Contabilidade e Sust..."
8,2015-03-23,PRODUTO VERDE: UMA ANÁLISE A PARTIR DO ECODESI...,Este estudo objetivou analisar através do uso ...,Estratégia ambiental. Desempenho ambiental do ...,2,Isabel Joselita Barbosa da Rocha Alves,Lúcia Santana de Freitas,,,,,Universidade Estadual da Paraíba,Universidade Federal de Campina Grande,,,,,https://reunir.revistas.u